# ResNet Models

**Atención**: No es necesario ejecutar ninguna de estas celdas debido a su excesivo tiempo de ejecución. El propósito de este Notebook es la visualización de los diferentes tamaños y precisiones dependiendo de la red y sus cuantizaciones.

## ResNet 18

#### Libraries and dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np
import os
import matplotlib.pyplot as plt

from torch.ao.quantization.fake_quantize import FakeQuantize
from torch.ao.quantization.observer import HistogramObserver, MovingAverageMinMaxObserver, MovingAveragePerChannelMinMaxObserver
from torch.ao.quantization.qconfig import QConfig
import torch.ao.quantization as quantization

#### Dataset and loaders

In [2]:
data_dir = './ImageNet1k/imagenet1k'
transform = transforms.Compose(
                [
                    #transforms.Resize(224),
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])

train_datasets = datasets.ImageFolder(os.path.join(data_dir), transform)
train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=6)
print('Validation dataset size:', len(train_datasets))

class_names = train_datasets.classes
print('The number of classes:', len(class_names))

Validation dataset size: 100001
The number of classes: 1000


In [3]:
data_dir = './Small-ImageNet1k/ILSVRC2012_img_val_subset'
transform = transforms.Compose(
                [
                    #transforms.Resize(224),
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])

val_datasets = datasets.ImageFolder(os.path.join(data_dir), transform)
val_dataloader = torch.utils.data.DataLoader(val_datasets, batch_size=16, shuffle=True, num_workers=6)
print('Validation dataset size:', len(val_datasets))

class_names = val_datasets.classes
print('The number of classes:', len(class_names))

Validation dataset size: 5000
The number of classes: 1000


#### ResNet Model

In [4]:
model = torchvision.models.resnet18(weights = torchvision.models.ResNet18_Weights.IMAGENET1K_V1, progress = True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### Fine-Tunning

Tras numerosas pruebas, se ha concluido que el proceso de finetunning es efectivo para mellorar la precisión de la red planteada pero no merece la pena en comparación al tiempo que lleva (+-2 horas). Por ello, se plantea dejar esta sección sin ejecutar si se están realizando pruebas sobre el resto del Notebook.

In [ ]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
set_parameter_requires_grad(model)

In [ ]:
model.fc = nn.Linear(512, 1000, bias = True)

In [ ]:
def train(optimizer, criteria, epoch, log_interval=len(train_dataloader.dataset)):
    model.train()
    
    for batch_idx, (images, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(images)
        loss = criteria(output, labels)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(images), len(train_dataloader.dataset), 100. * batch_idx / len(train_dataloader), loss.item()))

In [ ]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

epochs = 5
for epoch in range(epochs):
    train(optimizer, criteria, epoch)
    #validate()

#### Test

In [5]:
torch.save(model.state_dict() , './models/ResNet18/original_model.pth')

float_model_size = os.path.getsize('./models/ResNet18/original_model.pth') / 1024**2
print("Float model: {:.3f}MB".format(float_model_size))

Float model: 44.666MB


In [6]:
model = torchvision.models.resnet18(weights = torch.load('./models/ResNet18/original_model.pth', weights_only = True), progress = True)

/home/brais.martinez/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
def test(network, criteria):
    network.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (inputs, basic_labels) in enumerate(val_dataloader):

            labels = torch.zeros_like(basic_labels)
            for j in range(labels.shape[0]):
                labels[j] = int(class_names[basic_labels[j]])
            
            output = network(inputs)
            _, preds = torch.max(output, 1)
            
            test_loss += criteria(output, labels).item() * inputs.size(0)
            correct += preds.eq(labels.data.view_as(preds)).sum()
            
    test_loss /= len(val_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(val_dataloader.dataset), 0100. * correct / len(val_dataloader.dataset)))

In [8]:
test(model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.670281, Accuracy: 3143/5000 (62.86%)



#### Quantized model

In [9]:
import torchvision.models.quantization as qt
quant_model = qt.resnet18(weights = qt.ResNet18_QuantizedWeights.IMAGENET1K_FBGEMM_V1, quantize = True)
quant_model.eval()

/home/brais.martinez/.local/lib/python3.10/site-packages/torch/_utils.py:398: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  scales = torch.tensor(scales, dtype=torch.double, device=storage.device)


QuantizableResNet(
  (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.028605546802282333, zero_point=0, padding=(3, 3))
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): QuantizableBasicBlock(
      (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.016524722799658775, zero_point=0, padding=(1, 1))
      (bn1): Identity()
      (relu): Identity()
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.04645531252026558, zero_point=75, padding=(1, 1))
      (bn2): Identity()
      (add_relu): QFunctional(
        scale=0.03447607904672623, zero_point=0
        (activation_post_process): Identity()
      )
    )
    (1): QuantizableBasicBlock(
      (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.017180869355797768, zero_point=0, padding=(1, 1))
      (bn1): Iden

In [10]:
torch.save(quant_model.state_dict() , './models/ResNet18/original_quant_model.pth')

model_size = os.path.getsize('./models/ResNet18/original_quant_model.pth') / 1024**2
print("Float model: {:.3f}MB".format(model_size))

Float model: 11.293MB


In [11]:
test(quant_model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.689600, Accuracy: 3079/5000 (61.58%)



#### Post-Training Quantization

In [12]:
ptq_model_8bit = torch.quantization.quantize_dynamic(model, {nn.Linear, nn.Conv2d, nn.BatchNorm2d}, dtype=torch.qint8)
torch.save(ptq_model_8bit.state_dict(), './models/ResNet18/ptq_model_8bit.pth')

In [13]:
model_size = os.path.getsize('./models/ResNet18/ptq_model_8bit.pth') / 1024**2
print("Float model: {:.3f}MB".format(model_size))

Float model: 43.202MB


In [14]:
test(quant_model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.677776, Accuracy: 3098/5000 (61.96%)



## ResNet50

In [15]:
model = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1, progress = True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#### Test

In [16]:
torch.save(model.state_dict() , './models/ResNet50/original_model.pth')

float_model_size = os.path.getsize('./models/ResNet50/original_model.pth') / 1024**2
print("Float model: {:.3f}MB".format(float_model_size))

Float model: 97.793MB


In [17]:
model = torchvision.models.resnet50(weights = torch.load('./models/ResNet50/original_model.pth', weights_only = True), progress = True)

/home/brais.martinez/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
def test(network, criteria):
    network.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (inputs, basic_labels) in enumerate(val_dataloader):

            labels = torch.zeros_like(basic_labels)
            for j in range(labels.shape[0]):
                labels[j] = int(class_names[basic_labels[j]])
            
            output = network(inputs)
            _, preds = torch.max(output, 1)
            
            test_loss += criteria(output, labels).item() * inputs.size(0)
            correct += preds.eq(labels.data.view_as(preds)).sum()
            
    test_loss /= len(val_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(val_dataloader.dataset), 0100. * correct / len(val_dataloader.dataset)))

In [19]:
test(model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.316671, Accuracy: 3448/5000 (68.96%)



#### Quantized Model

In [20]:
import torchvision.models.quantization as qt
quant_model = qt.resnet50(weights = qt.ResNet50_QuantizedWeights.IMAGENET1K_FBGEMM_V1, quantize = True)
quant_model.eval()

QuantizableResNet(
  (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.02412003092467785, zero_point=0, padding=(3, 3))
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): QuantizableBottleneck(
      (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.010138723067939281, zero_point=0)
      (bn1): Identity()
      (conv2): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.01211862824857235, zero_point=0, padding=(1, 1))
      (bn2): Identity()
      (conv3): QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.018787238746881485, zero_point=59)
      (bn3): Identity()
      (relu): ReLU()
      (downsample): Sequential(
        (0): QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.03501461446285248, zero_point=76)
        (1): Identity()
      )
      (skip_add_relu)

In [21]:
torch.save(quant_model.state_dict() , './models/ResNet50/original_quant_model.pth')

model_size = os.path.getsize('./models/ResNet50/original_quant_model.pth') / 1024**2
print("Float model: {:.3f}MB".format(model_size))

Float model: 24.957MB


In [22]:
test(quant_model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.292659, Accuracy: 3492/5000 (69.84%)



#### Post-Training Quantization

In [23]:
ptq_model_8bit = torch.quantization.quantize_dynamic(model, {nn.Linear, nn.Conv2d, nn.BatchNorm2d}, dtype=torch.qint8)
torch.save(ptq_model_8bit.state_dict(), './models/ResNet50/ptq_model_8bit.pth')

In [24]:
model_size = os.path.getsize('./models/ResNet50/ptq_model_8bit.pth') / 1024**2
print("Float model: {:.3f}MB".format(model_size))

Float model: 91.934MB


In [25]:
test(quant_model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.322928, Accuracy: 3462/5000 (69.24%)



## ResNet 152

In [26]:
model = torchvision.models.resnet152(weights = torchvision.models.ResNet152_Weights.IMAGENET1K_V1, progress = True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#### Test

In [27]:
torch.save(model.state_dict() , './models/ResNet152/original_model.pth')

float_model_size = os.path.getsize('./models/ResNet152/original_model.pth') / 1024**2
print("Float model: {:.3f}MB".format(float_model_size))

Float model: 230.481MB


In [28]:
model = torchvision.models.resnet152(weights = torch.load('./models/ResNet152/original_model.pth', weights_only = True), progress = True)

/home/brais.martinez/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
def test(network, criteria):
    network.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (inputs, basic_labels) in enumerate(val_dataloader):

            labels = torch.zeros_like(basic_labels)
            for j in range(labels.shape[0]):
                labels[j] = int(class_names[basic_labels[j]])
            
            output = network(inputs)
            _, preds = torch.max(output, 1)
            
            test_loss += criteria(output, labels).item() * inputs.size(0)
            correct += preds.eq(labels.data.view_as(preds)).sum()
            
    test_loss /= len(val_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(val_dataloader.dataset), 0100. * correct / len(val_dataloader.dataset)))

In [30]:
test(model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.213060, Accuracy: 3591/5000 (71.82%)



#### Quantized Model

In [31]:
import torchvision.models.quantization as qt
quant_model = qt.resnet152(weights = qt.ResNet152_QuantizedWeights.IMAGENET1K_FBGEMM_V1, quantize = True)
quant_model.eval()

AttributeError: module 'torchvision.models.quantization' has no attribute 'resnet152'

No se puede realizar este apartado debido a que el módulo de quantization propio de Pytorch únicamente se encuentra para la ResNet18 y la ResNet50. El resto de ResNets no cuentan con cuantización propia de Pytorch, que es la que mejor resultados da, por lo que nos tendremos que conformar con la realizada a mano mediante los módulos de cuantización de 8 bits de Pytorch normal.

#### Post-Training Quantization

In [32]:
ptq_model_8bit = torch.quantization.quantize_dynamic(model, {nn.Linear, nn.Conv2d, nn.BatchNorm2d}, dtype=torch.qint8)
torch.save(ptq_model_8bit.state_dict(), './models/ResNet152/ptq_model_8bit.pth')

In [33]:
model_size = os.path.getsize('./models/ResNet152/ptq_model_8bit.pth') / 1024**2
print("Float model: {:.3f}MB".format(model_size))

Float model: 224.622MB


In [34]:
test(quant_model, nn.CrossEntropyLoss())


Test set: Avg. loss: 1.329480, Accuracy: 3489/5000 (69.78%)



# The End

In [35]:
print(chr(sum(range(ord(min(str(not())))))))

ඞ
